# Prismatic structures of Li2MnO3

In this notebook I've developed a small interactive tool to view the final geometries of the prismatic models of Li$_2$MnO$_3$. Once the binder has loaded, just execute the cell below and then use the drop down menu's to choose the `model`, `Li concentration` and DFT `functional`.

<em> In order to execute the cell: select it with and click `Shift+Enter`.</em>

In [3]:
# Execute this cell to visualize the structures and compare them to the original model.

import os
import json
import imolecule

from pymatgen import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from monty.io import zopen
from ipywidgets import interact, fixed, Dropdown

# Lists of various structural models, Li concentrations and functional settings
models = ["Li2MnO3_type", "inverted_p2_like", "202639_Li2MnO3_pristine",
"Li2MnO3_type_O_shifted", "inverted_p2_like_O_shifted", "symm_inverted_p2_like"]
li_concentrations = ["0", "0.25"]
functional_list = ["scan" , "pbeu_Mn3.9", "hse"]

# Set the atom sizes and colors for imolecule
element_properties={
    "Li": { "color": 0x66ff66, "radius": 0.85 },
    "O": { "color": 0xff0000, "radius": 0.7 },
    "Mn": { "color": 0xf00fff, "radius": 1.35 },
}

data_dir = os.path.join(os.getcwd(), "li2mno3_prism")

def get_energy(data_file):
    """
    Quick method for extracting the final energy from a calculation.
    """

    with zopen(data_file) as file:
        data = json.load(file)
        return data["output"]["final_energy"]

def visualize_prism(data_dir, model, li_conc, functional):
    
    conc_dir = str(li_conc) + "_li"
    
    if li_conc == "0":
        orig_path = os.path.join(data_dir, conc_dir, model, model + "_charged.json")
    else:
        orig_path = os.path.join(data_dir, conc_dir, model, model + ".json")
        
    orig_cat = Structure.from_file(orig_path)
    super_orig = orig_cat.copy()
    super_orig.make_supercell([2, 2, 2])
    super_orig.to("cif", "temp.cif")
    print()
    print("Original Structure")
    print("------------------")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)
    
    final_path = os.path.join(data_dir, conc_dir, model, functional + "_relax", "final_cathode.json")
    final_cat = Structure.from_file(final_path)
    if functional == "hse":
        spg = SpacegroupAnalyzer(final_cat)
        super_final = spg.get_conventional_standard_structure()
    else:
        super_final = final_cat.copy()
    super_final.make_supercell([2, 2, 2])
    super_final.to("cif", "temp.cif")
    print()
    print("Final Structure")
    print("---------------")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)
    
# Interactive function for visualizing the structures
interact(visualize_prism,
         data_dir=fixed(data_dir),
         model=models, 
         li_conc=Dropdown(options=li_concentrations,
                         description="Li concentration"),
         functional=functional_list);

interactive(children=(Dropdown(description='model', options=('Li2MnO3_type', 'inverted_p2_like', '202639_Li2Mn…

In [10]:
# Final energy analysis

for functional in ["scan" , "pbeu_Mn3.9"]:
    print(functional)
    print("-" * len(functional))
    print()
    print("75% charged:")
    print()
    
    for model in models:
        
        cat = Structure.from_file(
            os.path.join(data_dir, "0.25_li", model, functional + "_relax", "final_cathode.json")
        )
        print("\t" + model + " energy = " + str(get_energy(
            os.path.join(data_dir, "0.25_li", model, functional + "_relax", "data.json")
        )/len(cat)))
        
    print()
    print("100% charged:")
    print()
    
    for model in models:
        
        cat = Structure.from_file(
            os.path.join(data_dir, "0_li", model, functional + "_relax", "final_cathode.json")
        )
        print("\t" + model + " energy = " + str(get_energy(
            os.path.join(data_dir, "0_li", model, functional + "_relax", "data.json")
        )/len(cat)))
        
    print()
        

scan
----

75% charged:

	Li2MnO3_type energy = -9.919366303333334
	inverted_p2_like energy = -9.989952164444444
	202639_Li2MnO3_pristine energy = -9.900219832222222
	Li2MnO3_type_O_shifted energy = -10.00251505
	inverted_p2_like_O_shifted energy = -9.990443339444445
	symm_inverted_p2_like energy = -9.89415633574074

100% charged:

	Li2MnO3_type energy = -9.131679949999999
	inverted_p2_like energy = -9.209957917222221
	202639_Li2MnO3_pristine energy = -9.129517669444445
	Li2MnO3_type_O_shifted energy = -9.223961112222222
	inverted_p2_like_O_shifted energy = -9.212688115555556
	symm_inverted_p2_like energy = -9.24514534

pbeu_Mn3.9
----------

75% charged:

	Li2MnO3_type energy = -6.061436883888889
	inverted_p2_like energy = -6.056246078333333
	202639_Li2MnO3_pristine energy = -6.0614146077777775
	Li2MnO3_type_O_shifted energy = -6.141420398888889
	inverted_p2_like_O_shifted energy = -6.172984188333333
	symm_inverted_p2_like energy = -6.145198204259259

100% charged:

	Li2MnO3_type ener

In [11]:
# Quick interface function for comparing the result of two functionals

def compare_functional(data_dir, model, li_conc, functional1, functional2):
    
    conc_dir = str(li_conc) + "_li"
    
    final_path = os.path.join(data_dir, conc_dir, model, functional1 + "_relax", "final_cathode.json")
    final_cat = Structure.from_file(final_path)
    if functional1 == "hse":
        spg = SpacegroupAnalyzer(final_cat.as_ordered_structure())
        super_final = spg.get_conventional_standard_structure()
    else:
        super_final = final_cat.copy()
    super_final.make_supercell([2, 2, 2])
    super_final.to("cif", "temp.cif")
    print()
    print("Structure with " + functional1)
    print("---------------" + len(functional1)*"-")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)
    
    final_path = os.path.join(data_dir, conc_dir, model, functional2 + "_relax", "final_cathode.json")
    final_cat = Structure.from_file(final_path)
    if functional2 == "hse":
        spg = SpacegroupAnalyzer(final_cat.as_ordered_structure())
        super_final = spg.get_conventional_standard_structure()
    else:
        super_final = final_cat.copy()
    super_final.make_supercell([2, 2, 2])
    super_final.to("cif", "temp.cif")
    print()
    print("Structure with " + functional2)
    print("---------------" + len(functional2)*"-")
    print()
    imolecule.draw("temp.cif", element_properties=element_properties)

interact(compare_functional,
         data_dir=fixed(data_dir),
         model=models, 
         li_conc=li_concentrations,
         functional1=functional_list,
         functional2=functional_list);

interactive(children=(Dropdown(description='model', options=('Li2MnO3_type', 'inverted_p2_like', '202639_Li2Mn…